<a href="https://colab.research.google.com/github/embedashishg/SusAgri/blob/SusAgri/VLC_Participant_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is to test the implementation of the Rank Sort Algorithm for the VLC

Code has 3 parts -
  
  Part 1 - Open the Short listing sheet and read all the configuration variables. These variables are
  1. The Key of the Google sheet which holds the Registered Entry from the Google Form
  2. The total number of participants to be shortlisted
  3. Cut Off Ranking above which the participant shall be shortlisted

  Part 2 - Open the SourceWorkSheet - the key for which is sourced from the variable in part1.
  Then move all the entries from the SourceWorkSheet to a Worksheet in the Shortlist Sheet

  Part 3 - Run the Ranking algorithm on the basis of the configuration variables feteched in Part 1 and copy the shortlisted entries to Sorted Spreadsheet.   



In [2]:
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()

import gspread
from gspread.exceptions import WorksheetNotFound
from datetime import datetime

import pandas as pd
from pandas.core.indexers.utils import Any

gc = gspread.authorize(creds)

'''
This code is to test the implementation of the Rank Sort Algorithm for the VLC

Code has 3 parts -
  Part 1 - Open the Short listing sheet and read all the configuration variables. These variables are
  1. The Key of the Google sheet which holds the Registered Entry from the Google Form
  2. The total number of participants to be shortlisted
  3. Cut Off Ranking above which the participant shall be shortlisted

  Part 2 - Open the SourceWorkSheet - the key for which is sourced from the variable in part1.
          Then move all the entries from the SourceWorkSheet to a Worksheet in the Shortlist Sheet

  Part 3 - Run the Ranking algorithm on the basis of the configuration variables feteched in Part 1 and copy the shortlisted entries to Sorted Spreadsheet.
'''

''' Part 1 '''
#ShortListSheet - Replace this Key if the Shortlist Sheet changes
SpreadSheetFile = gc.open_by_key('1W7vVCSoiVXyw2oWrgsp2YeADrDd-WM0wR7EBDnxGxow')
ShortListVariableSheet = SpreadSheetFile.worksheet("Configurations")

#rows = ShortListVariableSheet.get_all_values()
#print (rows)

#Get the Variables to Run the Rank Sort

#Variable FormSheet Key to get the All the Participants
MatchCell = ShortListVariableSheet.find('Form Sheet Access Key')
#print (MatchCell)
#print (MatchCell.col, MatchCell.row)
AdjacentCell = ShortListVariableSheet.cell(MatchCell.row, MatchCell.col + 1)
FormSheetKey = AdjacentCell.value
print (FormSheetKey)

#Variable Cut-Off Score above which the Rank sort shall run  - Those below this cut-off shall be excluded
MatchCell = ShortListVariableSheet.find('Cut-Off Score')
#print (MatchCell)
#print (MatchCell.col, MatchCell.row)
AdjacentCell = ShortListVariableSheet.cell(MatchCell.row, MatchCell.col + 1)
CutOffScore =  int(AdjacentCell.value)
print ("Cut Off Score", CutOffScore)
#print (type(CutOffScore))

#Variable Number of Participants to Select
MatchCell = ShortListVariableSheet.find('Final Participants')
#print (MatchCell)
#print (MatchCell.col, MatchCell.row)
AdjacentCell = ShortListVariableSheet.cell(MatchCell.row, MatchCell.col + 1)
FinalParticipants = int(AdjacentCell.value)
print ("Total Participants to Select", FinalParticipants)
#NumberOfParticipantsToShortlist = ShortListVariableSheet.get()


142jW-fMPKTCSAN_YUCWXgmQMnpuAeS-tAeACvf3g0ig
Cut Off Score 45
Total Participants to Select 10


Part 2 - Open the SourceWorkSheet - the key for which is sourced from the variable in part1.
          Then move all the entries from the SourceWorkSheet to a Worksheet in the Shortlist Sheet

In [3]:

#Ensure Key is changed in case the Form and Google Sheet changes
SourceWorksheet = gc.open_by_key(FormSheetKey).sheet1

# get_all_values gives a list of rows.
AllEntries = SourceWorksheet.get_all_values()


#Write to the New Sheet in the current Sheet named 'AllParticipants'

try:
  AllParticipantsWorksheet = SpreadSheetFile.worksheet('AllParticipants')
  AllParticipantsWorksheet.clear()
  print ("AllParticipants Worksheet Exists - Opening It")
except:
  AllParticipantsWorksheet = SpreadSheetFile.add_worksheet(title="AllParticipants",rows=SourceWorksheet.row_count,cols=SourceWorksheet.col_count)
  print ("AllParticipants Worksheet Does not Exist - Creating It")


AllParticipantsWorksheet.update("A1", AllEntries)
print (AllParticipantsWorksheet.get_all_values())
# Convert to a DataFrame and render.

#AllParticipantsWorksheet.sort((1, 'asc'))


AllParticipants Worksheet Exists - Opening It
[['Timestamp', 'Email address', 'Score', 'Full Name', 'E-mail', 'Gender', 'Country', 'Highest Qualification', 'Work Experience in one of the following ?', 'How confident are you in your English language skills?', 'Please select the option that best describes your experience with social media campaigns.\nFacebook, Linkedin, X (Formerly known as Twitter) etc', 'Are you representing as an Individual or Organisation?', 'Individual Applicants  ', 'Name of Organisation', 'Designation', 'Organisation Detail', 'I agree that 100% attendance is mandatory for this training.', 'I agree to participate in the training, which will be conducted in the  GMT+2/GMT+5.30 time zone. (Please consider Daylight Saving changes at your location before selecting) '], ['13/09/2023 13:50:31', 'entoanshu@gmail.com', '30 / 50', 'Anshuman semwal', 'anshumanentomology@gmail.com', 'Male', 'India', 'PhD', 'Market Linkage (Supply/Value Chains) (Price Discovery)', 'Proficient'

 Part 3 - Run the Ranking algorithm on the basis of the configuration variables feteched in Part 1 and copy the shortlisted entries to Sorted Spreadsheet.


In [4]:

AllRows = AllParticipantsWorksheet.get_all_values()

#Ensure that the column header is maintained from the actual sheet and Not Panda else it will result in an error during Sorting by Timestamp!!
SelectionDFrame = pd.DataFrame(AllRows[1:], columns=AllRows[0])
#print (SelectionDFrame)
#Remove Delimiter only if Column type is string
if SelectionDFrame['Score'].dtype == 'object':
  if SelectionDFrame['Score'].str.contains('/').any():
   SelectionDFrame['Score'] = SelectionDFrame['Score'].str.split('/').str[0]

#Convert Score to Number if not one already
if SelectionDFrame['Score'].dtype != 'int64':
  SelectionDFrame['Score'] = SelectionDFrame['Score'].astype(int)


#print (SelectionDFrame)


First Sort by Ranking Score this is as per variable CutOffScore - acquired earlier from the Variable Sheet

In [ ]:
SelectionDFrame['Rank'] = SelectionDFrame['Score'].rank(ascending=False,method='max')

#print (SelectionDFrame['Score'], SelectionDFrame['Rank'])
#Remove Ranks below CutOffScore
#Select the ones which are above CutOffScore - remove the rest
print ("CutOff Score Is ", CutOffScore)
SelectionDFrame = SelectionDFrame[SelectionDFrame['Score'] >= CutOffScore]
#print (SelectionDFrame)



In [6]:
SelectionDFrame = SelectionDFrame.sort_values(by='Rank')
#print (SelectionDFrame)


In [7]:
#TopScorers = SelectionDFrame.head(CutOffScore)
#print (SelectionDFrame['Score'])

#Shortlist by Number of Participants to be selected FinalParticipants
SelectionDFrame = SelectionDFrame.head(FinalParticipants)
#print (SelectionDFrame)

Sort by Timestamp - in case multiple participants make a ranking shortlist - then select those on - First Come First Serve basis


In [ ]:
SelectionDFrame = SelectionDFrame.sort_values(by='Timestamp')

print ("Final Rows - ",SelectionDFrame.shape[0])

print ("Final Cols - ",SelectionDFrame.shape[1])

print (SelectionDFrame)



Create a New Worksheet where the shortlisted results shall appear. If it is already created open it else create a new one.

In [ ]:
try:
  FinalParticipantWorksheet = SpreadSheetFile.worksheet('FinalParticipants')
  FinalParticipantWorksheet.clear()
  print ("FinalParticipants Worksheet Exists - Opening It")
except:
  FinalParticipantWorksheet = SpreadSheetFile.add_worksheet(title="FinalParticipants",rows=SelectionDFrame.shape[0],cols=SelectionDFrame.shape[1])
  print ("FinalParticipants Worksheet Does not Exist - Creating It")
  print ("Write Final Data Frame to File")



Finally Write the Shortlisted Entries to the new Worksheet and close the selection process

In [21]:
  #Get Headers
  ResultHeaders = SelectionDFrame.columns.to_list()
  #print (ResultHeaders)

  #Get All Entries
  ShortListedEntries = SelectionDFrame.to_numpy().tolist()
  #print (ShortListedEntries)

  #Combine Headers and ShortListedEntries

  ShortListedEntries = [ResultHeaders] + ShortListedEntries

  #print (ShortListedEntries)
  #Write Final DataFrame to Worksheet
  FinalParticipantWorksheet.update(None,ShortListedEntries)

  #print ("All Done Ending Session and Closing File" )
  FinalParticipantWorksheet.spreadsheet.client.session.close()
  gc.session.close()
  #print ("Session Closed")


All Done Ending Session and Closing File
Session Closed
